In [1]:
from pyspark import SparkContext

In [2]:
sc= SparkContext()

### reduceBYKey y foldByKey() - son transformciones y no acciones

In [5]:
tupla= [('a',1),('b',2),('c',3), ('a',4),('b',5), ('c',6),('c',7)]

In [6]:
tupla

[('a', 1), ('b', 2), ('c', 3), ('a', 4), ('b', 5), ('c', 6), ('c', 7)]

In [8]:
pair_rdd= sc.parallelize(tupla) #pasamos a formato RDD

In [9]:
pair_rdd.collect()

[('a', 1), ('b', 2), ('c', 3), ('a', 4), ('b', 5), ('c', 6), ('c', 7)]

In [10]:
from operator import add

In [13]:
rdd_reduce= pair_rdd.reduceByKey(add) # Por cada clave nos ha hecho la suma delos valores

In [12]:
rdd_reduce.collect() 

[('a', 5), ('b', 7), ('c', 16)]

In [14]:
rdd_fold= pair_rdd.foldByKey(0,add)

In [15]:
rdd_fold.collect()

[('a', 5), ('b', 7), ('c', 16)]

### groupByKey agrupa los valores para cada clave - es una operación muy costosa

In [18]:
rdd_group= pair_rdd.groupByKey()

In [20]:
rdd_group.collect() #lista con unos iterables

[('a', <pyspark.resultiterable.ResultIterable at 0x7fc6bc3a15c0>),
 ('b', <pyspark.resultiterable.ResultIterable at 0x7fc6bc3a1320>),
 ('c', <pyspark.resultiterable.ResultIterable at 0x7fc6bc3a1438>)]

In [22]:
lista= [(x,list(y)) for x, y in rdd_group.collect()]

In [23]:
lista

[('a', [1, 4]), ('b', [2, 5]), ('c', [3, 6, 7])]

### combineByKey(createCombiner(),mergeValue(), mergeCombiners()) - parecido a agregate

In [25]:
rdd_suma_cuenta= pair_rdd.combineByKey(lambda x: (x,1),
                                       lambda x, y: (x[0]+y, x[1]+1),
                                       lambda x, y: (x[0], y[0], x[1]+ y[1]))
                                    

In [26]:
rdd_suma_cuenta.collect()

[('a', (5, 2)), ('b', (7, 2)), ('c', (16, 3))]

In [27]:
rdd_media= rdd_suma_cuenta.mapValues(lambda v: [v[0]/v[1]])

In [28]:
rdd_media.collect()

[('a', [2.5]), ('b', [3.5]), ('c', [5.333333333333333])]

## Transforamaciones para claves o valores

In [30]:
pair_rdd.collect()

[('a', 1), ('b', 2), ('c', 3), ('a', 4), ('b', 5), ('c', 6), ('c', 7)]

In [31]:
pair_rdd.keys().collect()

['a', 'b', 'c', 'a', 'b', 'c', 'c']

In [32]:
pair_rdd.values().collect()

[1, 2, 3, 4, 5, 6, 7]

In [33]:
pair_rdd.sortByKey().collect()

[('a', 1), ('a', 4), ('b', 2), ('b', 5), ('c', 3), ('c', 6), ('c', 7)]

In [34]:
# mapValues() - aplica la función sobre los valores
# flatMapValues() - aplica la función sobre los valores y crea una lista simplificad

In [35]:
pair_rdd.mapValues(lambda x: (x,x*10)).collect()

[('a', (1, 10)),
 ('b', (2, 20)),
 ('c', (3, 30)),
 ('a', (4, 40)),
 ('b', (5, 50)),
 ('c', (6, 60)),
 ('c', (7, 70))]

In [38]:
pair_rdd.flatMapValues(lambda x: (x,x*10)).collect()

[('a', 1),
 ('a', 10),
 ('b', 2),
 ('b', 20),
 ('c', 3),
 ('c', 30),
 ('a', 4),
 ('a', 40),
 ('b', 5),
 ('b', 50),
 ('c', 6),
 ('c', 60),
 ('c', 7),
 ('c', 70)]